<a href="https://play-iowa.neurodesk.org/hub/user-redirect/lab/tree/example-notebooks/books/structural_imaging/qsmxt_example.ipynb" target="_parent"><img src="https://img.shields.io/badge/launch-binder-579aca.svg?logo=" alt="Open In Binder"/>  </a>
<a href="https://colab.research.google.com/github/NeuroDesk/example-notebooks/blob/main/books/structural_imaging/qsmxt_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>   </a>

## QSMxT Example

Author: Ashley Stewart

Original paper: https://onlinelibrary.wiley.com/doi/10.1002/mrm.29048

## Setup Neurodesk

In [ ]:
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  os.environ["LD_PRELOAD"] = "";
  os.environ["APPTAINER_BINDPATH"] = "/content,/tmp,/cvmfs"
  os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
  os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

  !curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
  !chmod +x googlecolab_setup.sh
  !./googlecolab_setup.sh

  os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))


In [ ]:
# Output CPU information:
!cat /proc/cpuinfo | grep 'vendor' | uniq
!cat /proc/cpuinfo | grep 'model name' | uniq

# QSMxT Interactive Notebook

This interactive notebook estimates Quantitative Susceptibility Maps (QSMs) for two gradient-echo (GRE) MRI acquisitions using [QSMxT](https://github.com/QSMxT/QSMxT) provided by the [Neurodesk](https://neurodesk.org) project.

## What is QSM?

QSM is a form of quantitative MRI (qMRI) that estimates the magnetic susceptibility distribution across an imaged object. Magnetic susceptibility is the degree to which a material becomes magnetised by an external magnetic field. Major contributors to susceptibility include iron, calcium, and myelin, with the susceptibility of water typically approximating a zero-reference, though it is slightly diamagnetic. Read more about QSM [here](https://onlinelibrary.wiley.com/doi/10.1002/mrm.25358).

## What is QSMxT?

[QSMxT](https://qsmxt.github.io) is a suite of tools for building and running automated pipelines for QSM that:

- is available open-source without any licensing required;
- is distributed as a software container making it straightforward to access and install (Neurodesk!)
- scales its processing to execute across many acquisitions through jobs parallelisation (using multiple processors or HPCs) provided by [Nipype](https://nipype.readthedocs.io);
- automates steps that usually require manual intervention and scripting, including:
  - DICOM to [BIDS](https://bids-specification.readthedocs.io/en/stable/index.html) conversion;
  - QSM reconstruction using a range of algorithms;
  - segmentation using [FastSurfer](https://github.com/Deep-MI/FastSurfer);
  - group space generation using [ANTs](https://github.com/ANTsX/ANTs);
  - export of susceptibility statistics by subject and region of interest (ROI) to CSV.

![image.png](../images/image.png)

## How do I access QSMxT?

There are a few ways you can access QSMxT:

 - **This notebook**: You can access QSMxT in this notebook right now!
   - If you are running this on a Neurodesk Play instance, you can upload your own data into the sidebar via drag-and-drop.
 - **Neurodesktop**: QSMxT is in the applications menu of Neurodesktop.
   - On Neurodesk Play, upload your own data into the desktop via drag-and-drop.
   - On a local install of Neurodesk, bring any necessary files into the shared `~/neurodesktop-storage` directory
 - **Local install**: QSMxT can also be installed via the [Docker container](https://qsmxt.github.io/QSMxT/installation)
 - **HPC install**: QSMxT can also be installed via the [Singularity container](https://qsmxt.github.io/QSMxT/installation) for use on HPCs

# Download example DICOMs

Here, we download some example DICOMs from our OSF repository for QSMxT.

These data include GRE and T1-weighted acquisitions for one subject (duplicated to act as two subjects).

In [ ]:
!pip install osfclient
!osf -p ru43c clone . > /dev/null 2>&1
!tar xf osfstorage/dicoms-unsorted.tar
!rm -rf osfstorage/
!tree dicoms-unsorted | head
!echo -e "...\nThere are `ls dicoms-unsorted | wc -l` unsorted DICOMs in ./dicoms-unsorted/"

# Load QSMxT

To load QSMxT inside a notebook, we can use the available module system:

In [ ]:
import lmod
await lmod.load('qsmxt/6.4.4')
!qsmxt --version

# Data standardisation

QSMxT requires input data to conform to the [Brain Imaging Data Structure (BIDS)](https://bids.neuroimaging.io/).

Luckily, QSMxT also provides scripts that can convert unorganised NIfTI or DICOM images to BIDS. If you are using NIfTI images and do not have DICOMs, see [nifti-convert](https://qsmxt.github.io/QSMxT/using-qsmxt/data-preparation#nifti-to-bids).

## Sort DICOMs

Before we can convert DICOMs to BIDS cleanly, we need to sort the DICOMs by subject, session and series.

We can sort the DICOMs using `dicom-sort`.

Note that this script relies on accurate DICOM header information. If your data is sorted incorrectly, you may need to manually correct the sorting, or sort the files yourself. Be sure to follow the folder structure shown below.

In [ ]:
!dicom-sort dicoms-unsorted dicoms-sorted

Now we can see clearly that there are two subjects, each with one session, each with three DICOM series:

In [ ]:
!tree dicoms-sorted -L 3

## Convert to BIDS

Now that the DICOMs are sorted, we can convert to BIDS using `dicom-convert`.

The DICOM to BIDS conversion must identify which series should be used for QSM reconstruction (T2*-weighted), and which series should be used for segmentation (T1-weighted). Because this information is not stored in the DICOM header, the user must provide it, or QSMxT can make a guess based on the `ProtocolName` field. By default, QSMxT assumes series matching any of the patterns in `['*qsm*', '*t2starw*']` are to be used for QSM, and series matching the pattern `['*t1w*']` are to be used for segmentation. If series cannot be identified, the user must do so. At minimum, at least one QSM series must be identified.

If QSMxT is run interactively, the user will be prompted to identify the relevant series'. However, because we are running QSMxT in a notebook, we disable the interactivity using `--auto_yes` and provide the missing information using command-line arguments (`--t1w_protocol_patterns` and `--qsm_protocol_patterns`). In this case, the T1-weighted scan requires identification, so we pass `--t1w_protocol_patterns "*mp2rage*"`:

In [ ]:
!dicom-convert dicoms-sorted bids \
    --t1w_protocol_patterns "*mp2rage*" \
    --auto_yes

In [ ]:
!tree bids

# Inspect input data

Here we define a function we will use to visualise NIfTI images so we can view some of the input data:

In [ ]:
!pip install ipyniivue==2.0.1

In [ ]:
from glob import glob
from ipyniivue import AnyNiivue

In [ ]:
nv_mag = AnyNiivue()
nv_mag.load_volumes([{"path": glob("bids/sub-*/ses-*/anat/*mag*nii*")[0]}])
nv_mag

In [ ]:
nv_phase = AnyNiivue()
nv_phase.load_volumes([{"path": glob("bids/sub-*/ses-*/anat/*phase*nii*")[0]}])
nv_phase

In [ ]:
nv_t1w = AnyNiivue()
nv_t1w.load_volumes([{"path": glob("bids/sub-*/ses-*/anat/*T1w*nii*")[0]}])
nv_t1w

# Run QSMxT

We are now ready to run QSMxT! We will generate susceptibility maps and segmentations, and export analysis CSVs to file.

The usual way of running QSMxT is to use `qsmxt bids_dir output_dir`. This will launch an interactive command-line interface (CLI) to setup your desired pipelines. However, since we are running this in a notebook, we need to use command-line arguments to by-pass the interface and execute a pipeline.

But first, let's consider our pipeline settings. For QSM reconstruction, QSMxT provides a range of sensible defaults fit for different purposes. We can list the premade QSM pipelines using `--list_premades`. For the full pipeline details used for each premade pipeline, see [qsm_pipelines.json](https://github.com/QSMxT/QSMxT/blob/master/qsm_pipelines.json).

In [ ]:
!qsmxt --list_premades

For this demonstration, we will go with the `fast` pipeline. To export segmentations and analysis results, we will use `--do_segmentation` and `--do_analysis`. The `--auto_yes` option avoid the interactive CLI interface that cannot be used in a notebook:

In [ ]:
!qsmxt bids qsm \
    --premade fast \
    --do_qsm \
    --do_segmentation \
    --do_analysis \
    --auto_yes

# View results

Let's have a look at the generated `qsm` folder:

In [ ]:
!tree qsm/ -L 1 --dirsfirst

The `references.txt` file contains a list of all the algorithms used and relevant citations:

In [ ]:
!cat qsm/references.txt

Let's view one of the QSM results:

## QSM results

In [ ]:
!tree qsm/qsm -L 1 --dirsfirst

In [ ]:
show_nii(glob("qsm/qsm/*.nii*")[0], cmap='gray', vmin=-0.15, vmax=+0.15, interpolation='nearest')

## Segmentations

Segmentations are generated in both the QSM space and the T1-weighted space. Transformations are also made available.

In [ ]:
!tree qsm/segmentations

In [ ]:
show_nii(glob("qsm/segmentations/qsm/*.nii*")[0], cmap='terrain', vmin=0, vmax=96, interpolation='nearest')

## Analysis CSVs

Here, we can see CSV files have been exported containing susceptibility values in regions of interest for each subject:

In [ ]:
!tree qsm/analysis

Here we will load the CSVs, inspect the data and generate figures:

In [ ]:
!pip install seaborn numpy nibabel pandas matplotlib

In [ ]:
# import modules
import numpy as np
import nibabel as nib
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from glob import glob

The raw CSV files use names from FreeSurfer as exported by FastSurfer. The full list of regions is available [here](https://github.com/QSMxT/QSMxT/blob/master/aseg_labels.csv).

In [ ]:
pd.read_csv(glob("qsm/analysis/sub-1*.csv")[0])

We will select a subset of these ROIs and give them more readable names:

In [ ]:
# define regions of interest
# see https://github.com/QSMxT/QSMxT/blob/master/aseg_labels.csv for a full list
rois = { 
    "Thalamus" : [9, 10, 48, 49],
    "Pallidum" : [12, 13, 52, 53],
    "Caudate" : [11, 50],
    "Putamen" : [12, 51],
    "Brain stem" : [16],
    "CSF" : [24, 122, 257, 701],
    "White matter" : [2, 7, 41, 46, 177]
}
roi_names = { value: key for key in rois for value in rois[key] }
roi_ids = [value for roi in rois.values() for value in roi]

In [ ]:
# load a reconstruction
qsm = nib.load(glob("qsm/qsm/*.nii*")[0]).get_fdata().flatten()
seg = nib.load(glob("qsm/segmentations/qsm/*.nii*")[0]).get_fdata().flatten()

In [ ]:
# retain only the rois
qsm = qsm[np.isin(seg, roi_ids)]
seg = seg[np.isin(seg, roi_ids)]

In [ ]:
# convert to a dataframe for plotting purposes
seg = pd.Series(seg).map(roi_names)
data = pd.DataFrame({ 'qsm' : qsm, 'seg' : seg })

In [ ]:
# summarise data by region including the average and standard deviation
data.groupby('seg')['qsm'].agg(['mean', 'std']).sort_values('mean').round(decimals=3)

In [ ]:
medians = data.groupby('seg')['qsm'].median().sort_values()
order = medians.index

In [ ]:
# plot
fig = plt.figure()
ax = sns.boxplot(data, y='qsm', x='seg', fliersize=0, color='lightblue', order=order)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)
ax.set_ylim(-0.2, 0.3)
ax.axhline(y=0, color='pink', linestyle='-', linewidth=1, zorder=-1)
ax.set_xlabel("Region of interest")
ax.set_ylabel("Susceptibility (ppm)")
ax.set_title("QSM")
plt.show()